# scANVI make updated model with neutrophils
- Reran scvi without study as a categorical covariate as this currently isn't handled by scArches 
'''./run_scvi_adsaves.py --n-hvg 7500 \
            --remove-from-hvg feature_list/cc_genes.csv \
            --batch donorID_unified \
            --min-batch-size 0 \
            --continuous log1p_n_counts,percent_mito \
            --train-reference \


# Import and settings

In [1]:
%load_ext autoreload
%autoreload 2

import os, re, gc, joblib

import numpy as np
import numpy_groupies as npg
import scipy.sparse as sp
import pandas as pd
from sklearn.preprocessing import minmax_scale

import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.colors import ListedColormap, LogNorm

import seaborn as sn

import anndata

import scanpy as sc
import sctk as sk

import torch
import scvi


from collections import Counter

import numpy as np
from sklearn.neighbors import KNeighborsTransformer

Global seed set to 0


In [2]:
rcParams["pdf.fonttype"] = 42
rcParams["ps.fonttype"] = 42
expr_cmap = sk.expression_colormap()
np.set_printoptions(linewidth=150)
sc.settings.verbosity = 0
sk.set_figsize((4, 4))
#torch.cuda.set_device(1)

#PYTHON = "/software/team205/miniconda3/envs/rapids-torch/bin/python"

#%cd ~/scratch/20220125_digestive_tract_integration/v2/

# Define functions

In [3]:
def weighted_knn_trainer(train_adata, train_adata_emb, label_key, n_neighbors=50):
    """Trains a weighted KNN classifier on ``train_adata``.
    Parameters
    ----------
    train_adata: :class:`~anndata.AnnData`
        Annotated dataset to be used to train KNN classifier with ``label_key`` as the target variable.
    train_adata_emb: str
        Name of the obsm layer to be used for calculation of neighbors. If set to "X", anndata.X will be
        used
    label_key: str
        Name of the column to be used as target variable (e.g. cell_type) in ``train_adata`` and ``query_adata``.
    n_neighbors: int
        Number of nearest neighbors in KNN classifier.
    """
    print(
        f"Weighted KNN with n_neighbors = {n_neighbors} ... ",
        end="",
    )
    k_neighbors_transformer = KNeighborsTransformer(
        n_neighbors=n_neighbors,
        mode="distance",
        algorithm="brute",
        metric="euclidean",
        n_jobs=-1,
    )
    if train_adata_emb == "X":
        train_emb = train_adata.X
    elif train_adata_emb in train_adata.obsm.keys():
        train_emb = train_adata.obsm[train_adata_emb]
    else:
        raise ValueError(
            "train_adata_emb should be set to either 'X' or the name of the obsm layer to be used!"
        )
    k_neighbors_transformer.fit(train_emb)
    return k_neighbors_transformer

In [4]:
def weighted_knn_transfer(
    query_adata,
    query_adata_emb,
    ref_adata_obs,
    label_keys,
    knn_model,
    threshold=1,
    pred_unknown=False,
    mode="package",
):
    """Annotates ``query_adata`` cells with an input trained weighted KNN classifier.
    Parameters
    ----------
    query_adata: :class:`~anndata.AnnData`
        Annotated dataset to be used to queryate KNN classifier. Embedding to be used
    query_adata_emb: str
        Name of the obsm layer to be used for label transfer. If set to "X",
        query_adata.X will be used
    ref_adata_obs: :class:`pd.DataFrame`
        obs of ref Anndata
    label_keys: str
        Names of the columns to be used as target variables (e.g. cell_type) in ``query_adata``.
    knn_model: :class:`~sklearn.neighbors._graph.KNeighborsTransformer`
        knn model trained on reference adata with weighted_knn_trainer function
    threshold: float
        Threshold of uncertainty used to annotating cells as "Unknown". cells with
        uncertainties higher than this value will be annotated as "Unknown".
        Set to 1 to keep all predictions. This enables one to later on play
        with thresholds.
    pred_unknown: bool
        ``False`` by default. Whether to annotate any cell as "unknown" or not.
        If `False`, ``threshold`` will not be used and each cell will be annotated
        with the label which is the most common in its ``n_neighbors`` nearest cells.
    mode: str
        Has to be one of "paper" or "package". If mode is set to "package",
        uncertainties will be 1 - P(pred_label), otherwise it will be 1 - P(true_label).
    """
    if not type(knn_model) == KNeighborsTransformer:
        raise ValueError(
            "knn_model should be of type sklearn.neighbors._graph.KNeighborsTransformer!"
        )

    if query_adata_emb == "X":
        query_emb = query_adata.X
    elif query_adata_emb in query_adata.obsm.keys():
        query_emb = query_adata.obsm[query_adata_emb]
    else:
        raise ValueError(
            "query_adata_emb should be set to either 'X' or the name of the obsm layer to be used!"
        )
    top_k_distances, top_k_indices = k_neighbors_transformer.kneighbors(X=query_emb)

    stds = np.std(top_k_distances, axis=1)
    stds = (2.0 / stds) ** 2
    stds = stds.reshape(-1, 1)

    top_k_distances_tilda = np.exp(-np.true_divide(top_k_distances, stds))

    weights = top_k_distances_tilda / np.sum(
        top_k_distances_tilda, axis=1, keepdims=True
    )
    cols = ref_adata_obs.columns[ref_adata_obs.columns.str.startswith(label_keys)]
    uncertainties = pd.DataFrame(columns=cols, index=query_adata.obs_names)
    pred_labels = pd.DataFrame(columns=cols, index=query_adata.obs_names)
    for i in range(len(weights)):
        for j in cols:
            y_train_labels = ref_adata_obs[j].values
            unique_labels = np.unique(y_train_labels[top_k_indices[i]])
            best_label, best_prob = None, 0.0
            for candidate_label in unique_labels:
                candidate_prob = weights[
                    i, y_train_labels[top_k_indices[i]] == candidate_label
                ].sum()
                if best_prob < candidate_prob:
                    best_prob = candidate_prob
                    best_label = candidate_label

            if pred_unknown:
                if best_prob >= threshold:
                    pred_label = best_label
                else:
                    pred_label = "Unknown"
            else:
                pred_label = best_label

            if mode == "package":
                uncertainties.iloc[i][j] = (max(1 - best_prob, 0))

            else:
                raise Exception("Inquery Mode!")

            pred_labels.iloc[i][j] = (pred_label)

    print("finished!")

    return pred_labels, uncertainties

# Load data

In [5]:
ad_output = sc.read_h5ad('/home/jupyter/neutrophils/pooled_healthy.no_doublets.annot_all_levels.updated_donorID.20230126.metadata_update.withneutrophils.scvi_output.20240125-ad1.h5ad')

In [6]:
adata = sc.read_h5ad('/home/jupyter/neutrophils/pooled_healthy.no_doublets.annot_all_levels.updated_donorID.20230126.metadata_update.withneutrophils.20240125.h5ad')

In [7]:
adata

AnnData object with n_obs × n_vars = 1079134 × 36601
    obs: 'log1p_n_counts', 'percent_mito', 'level_1_annot', 'level_3_annot', 'donorID_unified'
    var: 'gene_ids', 'feature_type', 'mito', 'ribo', 'hb', 'n_counts', 'n_counts_raw', 'n_counts_spliced', 'n_counts_unspliced', 'n_cells', 'n_cells_raw', 'n_cells_spliced', 'n_cells_unspliced', 'cc', 'ig', 'tcr'

In [8]:
ad_output

AnnData object with n_obs × n_vars = 1079134 × 7280
    obs: 'log1p_n_counts', 'percent_mito', 'level_1_annot', 'level_3_annot', 'donorID_unified', '_scvi_batch', '_scvi_labels'
    var: 'gene_ids', 'feature_type', 'mito', 'ribo', 'hb', 'n_counts', 'n_counts_raw', 'n_counts_spliced', 'n_counts_unspliced', 'n_cells', 'n_cells_raw', 'n_cells_spliced', 'n_cells_unspliced', 'cc', 'ig', 'tcr', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'hvg'
    obsm: '_scvi_extra_continuous_covs'

In [9]:
adata = adata[:, adata.var_names.isin(ad_output.var_names)].copy()

In [10]:
adata

AnnData object with n_obs × n_vars = 1079134 × 7280
    obs: 'log1p_n_counts', 'percent_mito', 'level_1_annot', 'level_3_annot', 'donorID_unified'
    var: 'gene_ids', 'feature_type', 'mito', 'ribo', 'hb', 'n_counts', 'n_counts_raw', 'n_counts_spliced', 'n_counts_unspliced', 'n_cells', 'n_cells_raw', 'n_cells_spliced', 'n_cells_unspliced', 'cc', 'ig', 'tcr'

In [11]:
adata.obs.level_1_annot.unique()

['Endothelial', 'Myeloid', 'Epithelial', 'T and NK cells', 'Mesenchymal', 'B and B plasma', 'Neural']
Categories (7, object): ['B and B plasma', 'Endothelial', 'Epithelial', 'Mesenchymal', 'Myeloid', 'Neural', 'T and NK cells']

In [12]:
scvi_ref = scvi.model.SCVI.load("/home/jupyter/neutrophils/pooled_healthy.no_doublets.annot_all_levels.updated_donorID.20230126.metadata_update.withneutrophils.scvi_output.20240125.vae/",adata)

INFO     File /home/jupyter/neutrophils/pooled_healthy.no_doublets.annot_all_levels.updated_d
         onorID.20230126.metadata_update.withneutrophils.scvi_output.20240125.vae/model.pt   
         already downloaded                                                                  


# Process

In [13]:
scanvi_ref = scvi.model.SCANVI.from_scvi_model(
    scvi_ref, unlabeled_category="Unknown", labels_key="level_1_annot"
)

In [14]:
scanvi_ref.train(max_epochs=20, n_samples_per_label=100)

INFO     Training for 20 epochs.                                                             


/opt/conda/envs/scvi/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (ElboMetric). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_no_full_state`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/20:   0%|          | 0/20 [00:00<?, ?it/s]

/opt/conda/envs/scvi/lib/python3.9/site-packages/scvi/distributions/_negative_binomial.py:61: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/jupyter/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at  ../aten/src/ATen/native/cuda/jit_utils.cpp:860.)
  + torch.lgamma(x + theta)


Epoch 20/20: 100%|██████████| 20/20 [57:30<00:00, 172.52s/it, loss=882, v_num=1]


In [15]:
scanvi_ref.save(
    "/home/jupyter/Annotations_v3/updated_models_20240131/broad_annot_withNeutrophils_scANVI20240131/model.pt"
)